In [1]:
import xarray as xr
import numpy as np
import gsw as gsw
import matplotlib.pyplot as plt
import sys as sys
from oceanmixedlayers import oceanmixedlayers
import time
try:
    from Paths_BR import RG_T_File, RG_S_File, Save_File_Path
except:
    from Paths import RG_T_File, RG_S_File, Save_File_Path


In [2]:
Do_Temperature = True
Do_Salinity = True

time_var = 'TIME'
lat_var = 'LATITUDE'
lon_var = 'LONGITUDE'
z_var = 'PRESSURE'

T_hndl = xr.open_dataset(RG_T_File,decode_times=False)
PTemp_discrete = (T_hndl.ARGO_TEMPERATURE_MEAN + 
                  T_hndl.ARGO_TEMPERATURE_ANOMALY).transpose(z_var,time_var,lat_var,lon_var)
P_discrete = T_hndl[z_var].values

S_hndl = xr.open_dataset(RG_S_File,decode_times=False)
Salt_discrete = (S_hndl.ARGO_SALINITY_MEAN + 
                 S_hndl.ARGO_SALINITY_ANOMALY).transpose(z_var,time_var,lat_var,lon_var)

Label=''
LT = T_hndl[time_var].size
LY = T_hndl[lat_var].size
LX = T_hndl[lon_var].size
NZ = T_hndl[z_var].size
HNDL = T_hndl


In [3]:
#Create time indexes using pandas.  These are monthly averaged values starting from January 2004, with
# special consideration for leap-year.

import pandas as pd

YRS = 2004+np.floor(HNDL.TIME.values/12)
MOS = np.mod(HNDL.TIME.values,12)+0.5
DAS = np.ones(len(MOS))
HR = np.zeros(len(MOS))
for ii in [0,2,4,6,8,10,]:
    DAS[ii::12]=15
    HR[ii::12]=12
for ii in [3,5,7,9,11,]:
    DAS[ii::12]=15
    HR[ii::12]=0
for ii in range(1,len(MOS),12):
    if (np.mod(YRS[ii],4)==0):
        DAS[ii]=14
        HR[ii]=12
    else:
        DAS[ii]=14
        HR[ii]=0
Dates = (pd.to_datetime({'year':YRS,
                         'month':MOS,
                         'day':DAS,
                         'hour':HR
                     }))

print(Dates)

0     2004-01-15 12:00:00
1     2004-02-14 12:00:00
2     2004-03-15 12:00:00
3     2004-04-15 00:00:00
4     2004-05-15 12:00:00
              ...        
197   2020-06-15 00:00:00
198   2020-07-15 12:00:00
199   2020-08-15 00:00:00
200   2020-09-15 12:00:00
201   2020-10-15 00:00:00
Length: 202, dtype: datetime64[ns]


In [4]:
#This creates the data for the multiple energy values using the pe_anomaly method.
# This will take 10-20 minutes per energy value, so that the whole block will take 1-2 hours to complete.

for EN in [1,10,25,100]:
    out = xr.Dataset(data_vars={'mld':(('time','lat','lon'), np.add(np.NaN,np.zeros([LT,LY,LX]))),
                                'pe_anom':(('time','lat','lon'), np.add(np.NaN,np.zeros([LT,LY,LX]))),
                               },
                     coords={'time': (('time'),Dates),
                             'lat': (('lat'),HNDL[lat_var].values),
                             'lon': (('lon'),HNDL[lon_var].values)
                            }
                    )


    TIME=time.time()

    for Option in [3,]:

        #/Choices-----------------------------------------------------------------------
        #
        # Option 1
        if Option==1:
            method='prho_threshold_003'  # potential density threshold w/ 0.03 criteria
            print('\r',method,' '*80,end='')
        # Option 2
        if Option==2:
            method='holtetalley'         # holte talley algorithm method
            print('\r',method,' '*80,end='')
        # Option 3
        if Option==3:
            method='pe_anomaly'          # pe anomaly
            energy = EN
            gradient=True
            print('\r',method,'with gradient and e=',energy,' '*80,end=' ')
        # Option 4
        if Option==4:
            method='pe_anomaly'          # pe anomaly
            energy = EN
            gradient=False
            print('\r',method,'with e=',energy,' '*80,end=' ')
        #Option 5
        if Option==5:
            method='delta_pe'            # delta pe
            energy = EN
            print('\r',method,'with e=',energy,' '*80,end=' ')
        print('\n')
        for ti in range(LT):


            print('\r',ti,time.time()-TIME,' '*80,end=' ')

            S = np.array(Salt_discrete[:,ti,...].values,dtype='float')
            T = np.array(PTemp_discrete[:,ti,...].values,dtype='float')

            P_i=np.append(0.,P_discrete)

            SZ=np.shape(S)
            S_i=np.zeros([SZ[0]+1]+list(SZ[1:]))
            S_i[0,...] = S[0,...] ; S_i[1:,...] = S
            T_i=np.zeros([SZ[0]+1]+list(SZ[1:]))
            T_i[0,...] = T[0,...] ; T_i[1:,...] = T
            P_i = np.broadcast_to(P_i,np.shape(T_i.T)).T

            Rho_i = gsw.density.rho_t_exact(S_i,T_i,P_i)
            Rho_c = 0.5*(Rho_i[:-1,...]+Rho_i[1:,...])
            CT_i = gsw.conversions.CT_from_rho(Rho_i,S_i,P_i)[0]
            Rho0_i = gsw.density.rho(S_i,CT_i,0)
            Rho0_c = 0.5*(Rho0_i[:-1,...]+Rho0_i[1:,...])
            CT_c = 0.5*(CT_i[:-1,...]+CT_i[1:,...])
            S_c = 0.5*(S_i[:-1,...]+S_i[1:,...])
            P_c = 0.5*(P_i[:-1,...]+P_i[1:,...])
            dP = (P_i[:-1,...]-P_i[1:,...])
            Z_i=np.zeros([SZ[0]+1]+list(SZ[1:]))
            Z_i[1:,...] = np.cumsum(dP*1.e4/(Rho_c*9.81),axis=0)

            Z_c = 0.5*(Z_i[:-1,...]+Z_i[1:,...])
            dZ = (Z_i[:-1,...]-Z_i[1:,...])
            dRho0dz_c = (Rho0_i[:-1,...]-Rho0_i[1:,...])/dZ

            #-----------------------------------------------------------------------------


            if method == 'prho_threshold_003':
                LBL = method
                MLD,_ = oceanmixedlayers.threshold(abs(Z_c),
                                                   Rho0_c,
                                                   delta=0.03,
                                                   ref=10
                                                  )
            elif method == 'holtetalley':
                LBL = method
                OUT = oceanmixedlayers.holtetalley(P_c,
                                                   S_c,
                                                   CT_c,
                                                   Rho0_c)
                MLD_p=OUT[2] #This is in pressure
                MLD = np.zeros(np.shape(MLD_p))
                DP = np.zeros(np.shape(MLD_p))
                for ii in range(NZ):
                    li = DP<MLD_p
                    Pfrac_togo = (MLD_p-DP)/-dP[ii,...]
                    DP+=-dP[ii,...]
                    lf = DP>MLD_p

                    MLD[(li)&(~lf)]+=dZ[ii,(li)&(~lf)]
                    MLD[(li)&(lf)]+=(dZ[ii,...]*Pfrac_togo)[(li)&(lf)]
            elif method == 'pe_anomaly':
                if gradient == True: 
                    LBL=method+'_grad'+str(energy)
                else:
                    LBL = method+'_'+str(energy)

                MLD = oceanmixedlayers.mld_pe_anomaly(Z_c,
                                                      dZ,
                                                      Rho0_c,
                                                      dRho0dz_c,
                                                      energy=energy,
                                                      gradient=gradient
                                                 )
            elif method=='delta_pe':
                LBL = method+'_'+str(energy)
                MLD = oceanmixedlayers.mld_delta_pe(P_c,
                                              dP,
                                              CT_c,
                                              S_c,
                                              energy=energy,
                                             )
            pe_anom = oceanmixedlayers.pe_anomaly(Z_c,
                                                  dZ,
                                                  Rho0_c,
                                                  dRho0dz_c,
                                                  depth=-MLD,
                                                  gradient=True
                                                 )
            out.mld[ti,...] = MLD
            out.pe_anom[ti,...] = pe_anom


        filename = Save_File_Path+'/'+LBL+'.nc'
        out.to_netcdf(filename,encoding={
            'mld': {'_FillValue': -9999},
            'pe_anom': {'_FillValue': -9999},
        })


 pe_anomaly with gradient and e= 1                                                                                  

 pe_anomaly with gradient and e= 10                                                                                  

 pe_anomaly with gradient and e= 25                                                                                  

 pe_anomaly with gradient and e= 100                                                                                  

 201 830.3393633365631                                                                                    

In [5]:
#This creates the data for the threshold and Holte and Talley methods
#The threshold method is much faster, but this may take an hour or so as well.

out = xr.Dataset(data_vars={'mld':(('time','lat','lon'), np.add(np.NaN,np.zeros([LT,LY,LX]))),
                            'pe_anom':(('time','lat','lon'), np.add(np.NaN,np.zeros([LT,LY,LX]))),
                           },
                 coords={'time': (('time'),Dates),
                         'lat': (('lat'),HNDL[lat_var].values),
                         'lon': (('lon'),HNDL[lon_var].values)
                        }
                )


TIME=time.time()

for Option in [1,2]:

    #/Choices-----------------------------------------------------------------------
    #
    # Option 1
    if Option==1:
        method='prho_threshold_003'  # potential density threshold w/ 0.03 criteria
        print('\r',method,' '*80,end='')
    # Option 2
    if Option==2:
        method='holtetalley'         # holte talley algorithm method
        print('\r',method,' '*80,end='')
    # Option 3
    if Option==3:
        method='pe_anomaly'          # pe anomaly
        energy = EN
        gradient=True
        print('\r',method,'with gradient and e=',energy,' '*80,end=' ')
    # Option 4
    if Option==4:
        method='pe_anomaly'          # pe anomaly
        energy = EN
        gradient=False
        print('\r',method,'with e=',energy,' '*80,end=' ')
    #Option 5
    if Option==5:
        method='delta_pe'            # delta pe
        energy = EN
        print('\r',method,'with e=',energy,' '*80,end=' ')
    print('\n')
    for ti in range(LT):


        print('\r',ti,time.time()-TIME,' '*80,end=' ')

        S = np.array(Salt_discrete[:,ti,...].values,dtype='float')
        T = np.array(PTemp_discrete[:,ti,...].values,dtype='float')

        P_i=np.append(0.,P_discrete)

        SZ=np.shape(S)
        S_i=np.zeros([SZ[0]+1]+list(SZ[1:]))
        S_i[0,...] = S[0,...] ; S_i[1:,...] = S
        T_i=np.zeros([SZ[0]+1]+list(SZ[1:]))
        T_i[0,...] = T[0,...] ; T_i[1:,...] = T
        P_i = np.broadcast_to(P_i,np.shape(T_i.T)).T

        Rho_i = gsw.density.rho_t_exact(S_i,T_i,P_i)
        Rho_c = 0.5*(Rho_i[:-1,...]+Rho_i[1:,...])
        CT_i = gsw.conversions.CT_from_rho(Rho_i,S_i,P_i)[0]
        Rho0_i = gsw.density.rho(S_i,CT_i,0)
        Rho0_c = 0.5*(Rho0_i[:-1,...]+Rho0_i[1:,...])
        CT_c = 0.5*(CT_i[:-1,...]+CT_i[1:,...])
        S_c = 0.5*(S_i[:-1,...]+S_i[1:,...])
        P_c = 0.5*(P_i[:-1,...]+P_i[1:,...])
        dP = (P_i[:-1,...]-P_i[1:,...])
        Z_i=np.zeros([SZ[0]+1]+list(SZ[1:]))
        Z_i[1:,...] = np.cumsum(dP*1.e4/(Rho_c*9.81),axis=0)

        Z_c = 0.5*(Z_i[:-1,...]+Z_i[1:,...])
        dZ = (Z_i[:-1,...]-Z_i[1:,...])
        dRho0dz_c = (Rho0_i[:-1,...]-Rho0_i[1:,...])/dZ

        #-----------------------------------------------------------------------------


        if method == 'prho_threshold_003':
            LBL = method
            MLD,_ = oceanmixedlayers.threshold(abs(Z_c),
                                               Rho0_c,
                                               delta=0.03,
                                               ref=10
                                              )
        elif method == 'holtetalley':
            LBL = method
            OUT = oceanmixedlayers.holtetalley(P_c,
                                               S_c,
                                               CT_c,
                                               Rho0_c)
            MLD_p=OUT[2] #This is in pressure
            MLD = np.zeros(np.shape(MLD_p))
            DP = np.zeros(np.shape(MLD_p))
            for ii in range(NZ):
                li = DP<MLD_p
                Pfrac_togo = (MLD_p-DP)/-dP[ii,...]
                DP+=-dP[ii,...]
                lf = DP>MLD_p

                MLD[(li)&(~lf)]+=dZ[ii,(li)&(~lf)]
                MLD[(li)&(lf)]+=(dZ[ii,...]*Pfrac_togo)[(li)&(lf)]
        elif method == 'pe_anomaly':
            if gradient == True: 
                LBL=method+'_grad'+str(energy)
            else:
                LBL = method+'_'+str(energy)

            MLD = oceanmixedlayers.mld_pe_anomaly(Z_c,
                                                  dZ,
                                                  Rho0_c,
                                                  dRho0dz_c,
                                                  energy=energy,
                                                  gradient=gradient
                                             )
        elif method=='delta_pe':
            LBL = method+'_'+str(energy)
            MLD = oceanmixedlayers.mld_delta_pe(P_c,
                                          dP,
                                          CT_c,
                                          S_c,
                                          energy=energy,
                                         )
        pe_anom = oceanmixedlayers.pe_anomaly(Z_c,
                                              dZ,
                                              Rho0_c,
                                              dRho0dz_c,
                                              depth=-MLD,
                                              gradient=True
                                             )
        out.mld[ti,...] = MLD
        out.pe_anom[ti,...] = pe_anom


    filename = Save_File_Path+'/'+LBL+'.nc'
    out.to_netcdf(filename,encoding={
        'mld': {'_FillValue': -9999},
        'pe_anom': {'_FillValue': -9999},
    })


 prho_threshold_003                                                                                 

 holtetalley                                                                                              

 0 476.65122747421265                                                                                  

/net/bgr/Python/Packages/oceanmixedlayers/oceanmixedlayers/gradient.py:74: RuntimeWarning: All-NaN slice encountered
  dv_dc_max = np.nanmax(abs(dv_dc),axis=0)
/net/bgr/Python/Packages/oceanmixedlayers/oceanmixedlayers/extrema.py:40: RuntimeWarning: All-NaN slice encountered
  maxval = np.nanmax(value,axis=0)
/net/bgr/Python/Packages/oceanmixedlayers/oceanmixedlayers/gradient.py:208: RuntimeWarning: All-NaN slice encountered
  dv_dc_max = np.nanmax(abs(dv_dc),axis=0)
/net/bgr/Python/Packages/oceanmixedlayers/oceanmixedlayers/gradient.py:240: RuntimeWarning: All-NaN slice encountered
  closest = np.nanmin(abs(ml_values-cl_values),axis=0)
/net/bgr/Python/Packages/oceanmixedlayers/oceanmixedlayers/gradient.py:241: RuntimeWarning: All-NaN slice encountered
  maximum = np.nanmax(ml_values-cl_values,axis=0)
/net/bgr/Python/Packages/oceanmixedlayers/oceanmixedlayers/gradient.py:242: RuntimeWarning: All-NaN slice encountered
  minimum = np.nanmin(ml_values-cl_values,axis=0)
/net/bgr/Python/Pac

 5 548.5979866981506                                                                                   

/net/bgr/Python/Packages/oceanmixedlayers/oceanmixedlayers/gradient.py:177: RuntimeWarning: invalid value encountered in true_divide
  error = error/np.nansum(error,axis=0)


 201 3364.788346529007                                                                                   